In [2]:
import torch, platform
print("PyTorch:", torch.__version__, "| CUDA:", torch.cuda.is_available(), "| Python:", platform.python_version())


PyTorch: 2.8.0+cu126 | CUDA: True | Python: 3.12.11


In [3]:
!pip install -q transformers datasets evaluate scikit-learn accelerate
!pip -q install -U "transformers>=4.56.1" "datasets>=2.14" accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [4]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
# Set the path to the file you'd like to load
file_path = "1429_1.csv"
# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "datafiniti/consumer-reviews-of-amazon-products",
  file_path,
)
df.head(2)

/tmp/ipython-input-2879272884.py:6: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'consumer-reviews-of-amazon-products' dataset.


/usr/local/lib/python3.12/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman


In [5]:
df.keys()

Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')

In [6]:
#checking for NaN in the dataset
df.isnull().sum()

,0
id,0
name,6760
asins,2
brand,0
categories,0
keys,0
manufacturer,0
reviews.date,39
reviews.dateAdded,10621
reviews.dateSeen,0


In [7]:
#dropping unnecessary columns
columns_to_drop = [
    'id', 'dateAdded','reviews.didPurchase', 'brand','dateUpdated', 'asins', 'primaryCategories',
    'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber',
    'reviews.date', 'reviews.dateSeen','reviews.userCity',
       'reviews.userProvince'
    'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
    'reviews.sourceURLs',
       'reviews.userProvince','reviews.username','name', 'categories', 'sourceURLs','reviews.dateAdded'
]

# Get the columns that actually exist in the DataFrame
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

# Drop only the existing columns
df = df.drop(columns=existing_columns_to_drop)

In [8]:
df.keys()

Index(['reviews.doRecommend', 'reviews.rating', 'reviews.text',
       'reviews.title'],
      dtype='object')

In [9]:
df.head()

,reviews.doRecommend,reviews.rating,reviews.text,reviews.title
0,True,5.0,This product so far has not disappointed. My c...,Kindle
1,True,5.0,great for beginner or experienced person. Boug...,very fast
2,True,5.0,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.
3,True,4.0,I've had my Fire HD 8 two weeks now and I love...,Good!!!
4,True,5.0,I bought this for my grand daughter when she c...,Fantastic Tablet for kids


In [10]:
df.keys()

Index(['reviews.doRecommend', 'reviews.rating', 'reviews.text',
       'reviews.title'],
      dtype='object')

**IMPORTING NECECARY LIBUARIES**




In [11]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import torch

In [12]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)
from datasets import Dataset

In [13]:
# ------------------------------
# 1) Keep only the columns we need
# ------------------------------
# Common columns in Datafiniti Amazon reviews:
#   - 'reviews.text'      -> the review body
#   - 'reviews.rating'    -> star rating (often numeric 1..5)
needed_cols = []
if 'reviews.text' in df.columns:
    needed_cols.append('reviews.text')
if 'reviews.rating' in df.columns:
    needed_cols.append('reviews.rating')

assert len(needed_cols) == 2, f"Expected 'reviews.text' and 'reviews.rating' in df.columns, got: {df.columns.tolist()}"

df = df[needed_cols].copy()

In [14]:
# 2) Basic cleaning
# ------------------------------
def clean_text(s: str) -> str:
    s = re.sub(r'<br\s*/?>', ' ', str(s))      # remove HTML breaks
    s = re.sub(r'\s+', ' ', s).strip()         # normalize spaces
    return s

df['reviews.text'] = df['reviews.text'].fillna('').map(clean_text)
df['reviews.rating'] = pd.to_numeric(df['reviews.rating'], errors='coerce').astype('Int64')

# Drop empties & invalid ratings
df = df.dropna(subset=['reviews.rating'])
df = df[df['reviews.text'].str.len() > 0]
df = df[df['reviews.rating'].between(1, 5)]


In [15]:
# 3) Map stars → 3 sentiment classes
#    1-2 = 0 (negative), 3 = 1 (neutral), 4-5 = 2 (positive)
# ------------------------------
def map_star_to_label(star):
    if star in (1, 2):
        return 0
    if star == 3:
        return 1
    return 2  # 4 or 5

In [16]:
df['label'] = df['reviews.rating'].map(map_star_to_label).astype(int)
df['text']  = df['reviews.text']  # rename for HF Datasets convention

label_names = ['negative', 'neutral', 'positive']

print("Class counts:\n", df['label'].value_counts().sort_index().rename(index=dict(enumerate(label_names))))

# ------------------------------

Class counts:
 label
negative      812
neutral      1499
positive    32315
Name: count, dtype: int64


In [17]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset

In [18]:
# Cleaning data leaving only what we need

assert 'reviews.text' in df.columns and 'reviews.rating' in df.columns, df.columns
df = df[['reviews.text', 'reviews.rating']].copy()

In [19]:
# Pre-prossesing remove sigbola
def clean_text(s: str) -> str:
    s = re.sub(r'<br\s*/?>', ' ', str(s))
    s = re.sub(r'\s+', ' ', s).strip()
    return s

df['reviews.text'] = df['reviews.text'].fillna('').map(clean_text)
df['reviews.rating'] = pd.to_numeric(df['reviews.rating'], errors='coerce').astype('Int64')
df = df.dropna(subset=['reviews.rating'])
df = df[df['reviews.text'].str.len() > 0]
df = df[df['reviews.rating'].between(1, 5)]
df = df.drop_duplicates(subset=['reviews.text'])

In [20]:
# Map Review Points and labels (0=negative ,1=neutral ,2=positive)
def map_star_to_label(star):
    if star in (1, 2): return 0
    if star == 3:      return 1
    return 2

df['label'] = df['reviews.rating'].map(map_star_to_label).astype(int)
df['text']  = df['reviews.text']


In [21]:
# Training and Split
train_df, temp_df = train_test_split(
    df[['text','label']], test_size=0.2, random_state=42, stratify=df['label']
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df['label']
)

In [22]:
# Tokenizer + datasets loading model_name = "distilbert-base-uncased"
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True)).map(tokenize, batched=True)
val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True)).map(tokenize, batched=True)
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True)).map(tokenize, batched=True)

train_ds = train_ds.rename_columns({'label':'labels'})
val_ds   = val_ds.rename_columns({'label':'labels'})
test_ds  = test_ds.rename_columns({'label':'labels'})

data_collator = DataCollatorWithPadding(tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/27692 [00:00<?, ? examples/s]

Map:   0%|          | 0/3461 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [23]:
# Model ----------
id2label = {0: 'negative', 1: 'neutral', 2: 'positive'}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=3, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
#Evaluations Mtrics

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average='macro'),
        "f1_weighted": f1_score(labels, preds, average='weighted'),
    }


In [41]:
#Training Model with minimal arguments

training_args = TrainingArguments(
    output_dir="./distilbert-sentiment",
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=100,
    seed=42,
    report_to="none",
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss
100,0.004400
200,0.000000
300,0.023100
400,0.065400
500,0.017900
600,0.059400
700,0.052100
800,0.044700
900,0.027100
1000,0.048800


TrainOutput(global_step=25965, training_loss=0.01124365587087378, metrics={'train_runtime': 3439.4198, 'train_samples_per_second': 120.77, 'train_steps_per_second': 7.549, 'total_flos': 1.4175139971894768e+16, 'train_loss': 0.01124365587087378, 'epoch': 15.0})

In [43]:
#Evalutions Prints

print("\nValidation metrics:", trainer.evaluate(eval_dataset=val_ds))
print("\nTest metrics:", trainer.evaluate(eval_dataset=test_ds))

# Detailed report on test set
test_logits = trainer.predict(test_ds).predictions
test_preds = np.argmax(test_logits, axis=1)
print("\nClassification report (test):")
print(classification_report(test_df['label'], test_preds, target_names=['negative','neutral','positive'], digits=4))
print("Confusion matrix (rows=true, cols=pred):")
print(confusion_matrix(test_df['label'], test_preds))


Validation metrics: {'eval_loss': 0.8268521428108215, 'eval_accuracy': 0.9344120196475008, 'eval_f1_macro': 0.5374881787737106, 'eval_f1_weighted': 0.9239255366363719, 'eval_runtime': 3.8397, 'eval_samples_per_second': 901.366, 'eval_steps_per_second': 28.387, 'epoch': 15.0}

Test metrics: {'eval_loss': 0.8036422729492188, 'eval_accuracy': 0.938474870017331, 'eval_f1_macro': 0.593937170466189, 'eval_f1_weighted': 0.9298710179440045, 'eval_runtime': 3.8884, 'eval_samples_per_second': 890.34, 'eval_steps_per_second': 28.032, 'epoch': 15.0}

Classification report (test):
              precision    recall  f1-score   support

    negative     0.5946    0.5432    0.5677        81
     neutral     0.3750    0.1800    0.2432       150
    positive     0.9584    0.9836    0.9708      3231

    accuracy                         0.9385      3462
   macro avg     0.6427    0.5689    0.5939      3462
weighted avg     0.9246    0.9385    0.9299      3462

Confusion matrix (rows=true, cols=pred):
[[